In [ ]:
import pandas as pd, numpy as np, time
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
def auc(model, train, test): 
    return (metrics.roc_auc_score(y_train, model.predict_proba(train)[:,1]),
            metrics.roc_auc_score(y_test, model.predict_proba(test)[:,1]))

In [ ]:
train=pd.read_csv('X_train.csv')
test=pd.read_csv('X_test.csv')
y_train=np.array(pd.read_csv('y_train.csv'))
y_test=np.array(pd.read_csv('y_test.csv'))

### XGBoost

In [ ]:
model = xgb.XGBClassifier()
model.fit(train, np.array(y_train))

auc(model, train, test)

### LightGBM

In [ ]:
model2 = lgb.LGBMClassifier()
model2.fit(train, y_train)

auc(model2, train, test)

### Catboost

In [ ]:
clf = cb.CatBoostClassifier(iterations=30, silent=True)
clf.fit(train,y_train)

auc(clf, train, test)

## NEW TASK!
#### slides from Lecture 3 can help you :)

### Average of predictions

In [ ]:
### YOUR CODE HERE

### Weighted average of predictions

In [ ]:
### YOUR CODE HERE

### Bagging 

In [ ]:
### YOUR CODE HERE

### Stacking

In [ ]:
### YOUR CODE HERE